In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from pprint import pp
from que_onda import *
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
import soundfile as sf
import tensorflow as tf
import tensorflow_probability as tfp

np.set_printoptions(floatmode='unique')
dtype = tf.float32
n_levels = 10
wavelet_name = 'sym6'

wavelet = wavelets.by_name(wavelet_name, dtype)
filters = wavelets.dwt_filters(wavelet)


2025-04-22 14:11:06.834000: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-22 14:11:06.837322: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-22 14:11:06.849190: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-22 14:11:06.873202: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-22 14:11:06.873234: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-22 14:11:06.887690: I tensorflow/core/platform/cpu_feature_guard.cc:

In [3]:
# files = os.walk('data/mp3')
files = [
         ('data/mp3/MedleyDB_V1', [], [
                                   'AlexanderRoss_VelvetCurtain_MIX.mp3',
                                   'CelestialShore_DieForUs_MIX.mp3',
                                   'ClaraBerryAndWooldog_TheBadGuys_MIX.mp3',
                                   'ClaraBerryAndWooldog_WaltzForMyVictims_MIX.mp3',
                                   'Debussy_LenfantProdigue_MIX.mp3',
                                   'EthanHein_BluesForNofi_MIX.mp3'
                                   ]),
         ('data/mp3/MedleyDB_V2', [], [
                                   'Allegria_MendelssohnMovement1_MIX.mp3',
                                   'BarefootSisters_RedJetta_MIX.mp3',
                                   'CatMartino_IPromise_MIX.mp3',
                                   'Cayetana_MissThing_MIX.mp3',
                                   'DeclareAString_MendelssohnPianoTrio1Movement1_MIX.mp3',
                                   'DeclareAString_MendelssohnPianoTrio1Movement2_MIX.mp3'
                                  ])]

all_as = []
all_bs = []
all_cs = []
beat_lengths = []
min_beat_length = 9
max_beat_length = 63
max_lags = 188

# Logarithmic Decay with Shift: y = a * log(x + c) + b
def logarithmic_decay_shifted(x, a, b, c):
    return a * np.log(x + c) + b

vorbis_chunk_size = 128 * 1024

def write_channels_vorbis(filename, channels, samplerate):
    samples = tf.transpose(channels)
    np_samples = samples.numpy()
    with sf.SoundFile(filename, mode='w', channels=channels.shape[0], format='OGG', samplerate=samplerate, subtype='VORBIS') as file:
        for i in range(0, np_samples.shape[0], vorbis_chunk_size):
            chunk = np_samples[i:i+vorbis_chunk_size]
            file.write(chunk)

for dirpath, dirnames, filenames in files:
    for filename in filenames:
        basename, _ = os.path.splitext(filename)
        in_channels, samplerate = io.read_channels(os.path.join(dirpath, filename))
        # in_levels = wavelets.wavedec_same(filters, n_levels, in_channels)
        # in_frames = framing.levels_to_frames(in_levels)
        # frame_level_energies = analysis.energy_by_frame_and_level(in_frames)
        # frame_power = analysis.power_by_frame(frame_level_energies)

        # x_data = np.arange(0, max_lags + 1)
        # y_data = tfp.stats.auto_correlation(frame_power, max_lags=max_lags).numpy()

        # short_x_data = x_data[min_beat_length:max_beat_length]
        # short_y_data = y_data[min_beat_length:max_beat_length]

        # # peaks, peak_properties = find_peaks(y_data, prominence=0.006, width=2)
        # # plt.plot(x_data, y_data)
        # # plt.plot(peaks + min_beat_length, y_data[peaks], 'x')
        # # plt.savefig(f'temp/{basename}-lin.png')
        # # plt.close()

        # # shifted logarithmic decay model (Gemini)
        # try:
        #     popt_shifted, pcov_shifted = curve_fit(logarithmic_decay_shifted, x_data, y_data, p0=(-0.08439892383834803, 0.7862034695405674, 0.07126941591678249))
        # except Exception as e:
        #     print(f'!!! could not fit {basename}: {e}')
        #     continue
        # a_opt_shifted, b_opt_shifted, c_opt_shifted = popt_shifted
        # all_as.append(a_opt_shifted)
        # all_bs.append(b_opt_shifted)
        # all_cs.append(c_opt_shifted)
        # print(f'y = {a_opt_shifted} * e^(x+{c_opt_shifted}) + {b_opt_shifted}')

        # # Generate points for the fitted curves
        # y_fit_shifted = logarithmic_decay_shifted(short_x_data, *popt_shifted)
        # diff = short_y_data - y_fit_shifted

        # raw_peaks, peak_properties = find_peaks(diff, prominence=0.008, width=2)
        # peaks = raw_peaks + min_beat_length
        # beat_lengths.extend(peaks)
        # pp(peak_properties)

        # # Plot the data and the fits
        # plt.plot(short_x_data, y_fit_shifted)
        # plt.plot(x_data, y_data)
        # plt.plot(peaks, y_data[peaks], 'x')
        # plt.savefig(f'temp/{basename}-log.png')
        # plt.close()
        # print()

        # # beat_length = beats.find_length(frame_power)

        # # x = np.arange(1, 202)
        # # autocorrelation = tfp.stats.auto_correlation(frame_power, max_lags=200)
        # # np_autocorrelation = autocorrelation.numpy()
        # # beat_lengths.extend(peaks)

        # # https://www.geeksforgeeks.org/how-to-do-exponential-and-logarithmic-curve-fitting-in-python/
        # # logarithmic
        # # log_x = np.log(x)
        # # a, b = np.polyfit(log_x, np_autocorrelation, 1)
        # # est = (a * log_x) + b
        # # flattened = np_autocorrelation - est

        # # for beat_length in peaks:
        # #     n_frames = frame_power.shape[0]
        # #     n_beats = n_frames // beat_length
        # #     cut = frame_power[:n_beats * beat_length]
        # #     intrabeat_power = tf.reshape(cut, [-1, beat_length])
        # #     means = tf.reduce_mean(intrabeat_power, axis=0)
        # #     plt.plot(means)
        # #     plt.savefig(f'temp/{basename}-{beat_length}.png')
        # #     plt.close()


AlexanderRoss_VelvetCurtain_MIX: 44100Hz
num channels: 2
samplerate: 44100
CelestialShore_DieForUs_MIX: 44100Hz
num channels: 2
samplerate: 44100
ClaraBerryAndWooldog_TheBadGuys_MIX: 44100Hz
num channels: 2
samplerate: 44100
ClaraBerryAndWooldog_WaltzForMyVictims_MIX: 44100Hz
num channels: 2
samplerate: 44100
Debussy_LenfantProdigue_MIX: 44100Hz
num channels: 2
samplerate: 44100
EthanHein_BluesForNofi_MIX: 44100Hz
num channels: 2
samplerate: 44100
Allegria_MendelssohnMovement1_MIX: 44100Hz
num channels: 2
samplerate: 44100
BarefootSisters_RedJetta_MIX: 44100Hz
num channels: 2
samplerate: 44100
CatMartino_IPromise_MIX: 44100Hz
num channels: 2
samplerate: 44100
Cayetana_MissThing_MIX: 44100Hz
num channels: 2
samplerate: 44100
DeclareAString_MendelssohnPianoTrio1Movement1_MIX: 44100Hz
num channels: 2
samplerate: 44100
DeclareAString_MendelssohnPianoTrio1Movement2_MIX: 44100Hz
num channels: 2
samplerate: 44100


In [ ]:
(np.median(all_as), np.median(all_bs), np.median(all_cs))

In [ ]:
plt.hist(beat_lengths, bins=range(min(beat_lengths) - 1, max(beat_lengths) + 3, 1))

In [ ]:
# dataset, song, beat_length = 1, 'MusicDelta_Rock', 95                        # 49.5% (or 47, 20.5%)
# dataset, song, beat_length = 1, 'MusicDelta_Zeppelin', 94                      # 25.7% (or 23)
# dataset, song, beat_length = 1, 'MusicDelta_80sRock', 94                     # 61.5% (or 47, 64.6%)
# dataset, song, beat_length = 1, 'MusicDelta_Reggae', 94                      # 52.8% (or 47, 60.3%)
# dataset, song, beat_length = 1, 'MusicDelta_Hendrix',  94                    # 67.6%
# dataset, song, beat_length = 1, 'MusicDelta_InTheHalloftheMountainKing', 94  # 65.9%
# dataset, song, beat_length = 1, 'MusicDelta_Country2', 94                    # 71.7%
# dataset, song, beat_length = 1, 'MusicDelta_Shadows', 94                     # 26.0%
# dataset, song, beat_length = 1, 'MusicDelta_Vivaldi', 93                       # 27.2% (or 47, 35.8%)
# dataset, song, beat_length = 1, 'MusicDelta_Rockabilly', 94                  # 71.9%
# dataset, song, beat_length = 1, 'MusicDelta_Beethoven', 93                   # 24.4%
# dataset, song, beat_length = 1, 'MusicDelta_Beatles', 94                     # 44.3%
# dataset, song, beat_length = 2, 'Katzn_CharlieKnox', 42
# dataset, song, beat_length = 1, 'Creepoid_OldTree', 19
# dataset, song, beat_length = 1, 'TheSoSoGlos_Emergency', 29
# dataset, song, beat_length = 1, 'BigTroubles_Phantom', 21
# dataset, song, beat_length = 1, 'CelestialShore_DieForUs', 30
# dataset, song, beat_length = 1, 'TheDistricts_Vermont', 28

in_channels, samplerate = io.read_channels(f'data/MedleyDB_V{dataset}/{song}_MIX.mp3')


In [ ]:
name = 'sym6'
n_levels = 10

wavelet = wavelets.by_name(name, dtype)
filters = wavelets.dwt_filters(wavelet)
inverse_filters = wavelets.idwt_filters(wavelet)
in_levels = wavelets.wavedec_same(filters, n_levels, in_channels)
in_frames = framing.levels_to_frames(in_levels)
n_frames = in_frames[0].shape[0]


In [ ]:
frame_level_energies = analysis.energy_by_frame_and_level(in_frames)
frame_energies = analysis.energy_by_frame(frame_level_energies)
autocorrelation = tfp.stats.auto_correlation(frame_energies, max_lags=100)
np_autocorrelation = autocorrelation.numpy()

peaks, peak_properties = find_peaks(np_autocorrelation, height=0, prominence=0.05, width=1)
print(peaks)
first_peak = peaks[0]
print(f'first peak: {first_peak}')
print(f'properties: {peak_properties}')
print(f'At first peak: {np_autocorrelation[first_peak]}')
plt.plot(np_autocorrelation)
plt.plot(peaks, np_autocorrelation[peaks], 'x')


In [ ]:
offset = beats.find_offset(frame_energies, beat_length)
in_beats = beats.frames_to_beats(beat_length, in_frames, offset)
beat_level_energies = beats.energy_by_beat_and_level(in_beats)
beat_level_power = tf.sqrt(beat_level_energies)
distances = analysis.manhattan_distance_matrix(beat_level_power)
route = defrag.defrag(distances, time_limit=300)

In [ ]:
print(tf.reduce_max(beat_level_power, axis=0))

remixed = tf.gather(beat_level_power, route)
for l in tf.transpose(remixed):
    plt.plot(l)

In [ ]:
out_beats = beats.remix(in_beats, route)
out_frames = beats.beats_to_frames(out_beats)
out_levels = framing.frames_to_levels(out_frames)
out_channels = wavelets.waverec(inverse_filters, out_levels)
io.write_channels(f'temp/{song}-{name}-{beat_length}-sqrt.mp3', out_channels, samplerate)


In [ ]:
import soundfile as sf

sf.available_formats()